In [29]:
np.random.seed(42)
n_p = 28 # the period
fcst = np.random.poisson(lam=55, size=n_p) # two sampling "time series" from Poisson distribution
truth= np.random.poisson(lam=50, size=n_p) 
ds = InvtSim(j=0, fcst = fcst, truth = truth)
print(ds.ob_all_t())

    DtL          ot        sst          ip         wip         net  \
0   158  158.000000   0.000000    0.000000    0.000000    0.000000   
1   163   61.000000   0.000000  158.000000  158.000000    0.000000   
2   166   58.546910   8.546910  219.000000  219.000000    0.000000   
3   167    0.000000   7.477604  218.546910  119.546910   99.000000   
4   161    0.000000   7.477604  172.546910   58.546910  114.000000   
5   151   49.326119  24.873030  126.546910    0.000000  126.546910   
6   161   63.887373  23.760403  120.873030   49.326119   71.546910   
7   160   60.756927  25.517330  124.760403  113.213492   11.546910   
8   161   24.883792   9.401122  145.517330  124.644300   20.873030   
9   152   68.017555  30.418676  114.401122   85.640719   28.760403   
10  159   64.163656  31.582332  126.418676   92.901346   33.517330   
11  162   50.002177  28.584508  140.582332  132.181210    8.401122   
12  165   25.787561   5.372069  144.584508  114.165832   30.418676   
13  167   75.087156 

In [28]:
class InvtSim:
    def __init__(self, j, fcst, truth, L=3, alpha=0.95, period=28, h = 1, b = 19):
        self.h         = h
        self.b         = b 
        self.alpha     = alpha            # service level
        self.period    = period           # forecats horizon
        self.fcst      = np.array(fcst)   # forecasts
        self.truth     = np.array(truth)  # actual demand
        self.L         = L                # lead-time

        # store these information
        self.sst_l     = []  # 安全库存
        self.ot_l      = []  # order
        self.ipt_l     = []  # inventory position
        self.DtL_l     = []  # forecasted demands over the lead time
        self.net_l     = []  # net inventory
        self.wip_l     = []  # work in progress
        self.backlog_l = []  # backlog
        self.ch_l      = []  # holding costs
        self.cb_l      = []  # backlog costs

    def ob_ss_t(self, t, window=3):
        start = max(0, t - window)  # ensure the index available
        if t == 0:
            return 0  
        std_err = np.std(self.fcst[start:t] - self.truth[start:t])
        return norm.ppf(self.alpha) * std_err * np.sqrt(self.L)

    def ob_all_t(self):
        # initial inventory
        ip_t = self.truth[0] 
        net_t = self.truth[0]  # assumption that the initial net inventory is equal to the actual demand for the first period
        wip_t = 0
        backlog = 0 

        for t in range(self.period):
            # ss
            ss_t = self.ob_ss_t(t)
            self.sst_l.append(ss_t)
            # DtL
            DtL = np.sum(self.fcst[t:t + self.L])
            self.DtL_l.append(DtL)
            # o_t_L, o_t_1, wip_t, net_t
            o_t_L = self.ot_l[-self.L] if t >= self.L else 0
            wip_t = wip_t + (self.ot_l[-1] if t >= 1 else 0) - o_t_L
            net_t = net_t + o_t_L - self.truth[t]

            if net_t < 0:
                backlog += abs(net_t)  # record backlog
                net_t = 0
            else:
                backlog = max(0, backlog - net_t)  # backlog 

            # ip and order
            ip_t = wip_t + net_t # do not involve the backlog
            o_t = max(0, DtL + ss_t - ip_t + backlog) # consider the backlog

            # holding and backlogging cost
            ch_t = self.h * max(0, net_t)  
            cb_t = self.b * max(0, backlog)  

            # store all data
            self.ot_l.append(o_t)
            self.ipt_l.append(ip_t)
            self.net_l.append(net_t)
            self.wip_l.append(wip_t)
            self.backlog_l.append(backlog)
            self.ch_l.append(ch_t)
            self.cb_l.append(cb_t)

        return pd.DataFrame({
            'DtL': self.DtL_l, 'ot': self.ot_l, 'sst': self.sst_l,
            'ip': self.ipt_l, 'wip': self.wip_l, 'net': self.net_l,
            'backlog': self.backlog_l, 'ch': self.ch_l, 'cb': self.cb_l
        })